In [ ]:
import torch
import torch.nn as nn
import random
import pprint
from collections import defaultdict
import copy

from typing import List, Tuple, Dict, Any

# ─── 1. CONFIGURACIÓN ─────────────────────────────────────────────────────────

# Semilla para reproducibilidad
random.seed(42)
torch.manual_seed(42)

# Opciones de hiperparámetros (puedes expandir esto)
ACTIVATIONS = ['relu', 'leaky_relu', 'gelu']
POOLING_TYPES = [None, 'max', 'avg']
DROPOUT_PROB = [0.0, 0.1, 0.25]
CONV_OUT_CHANNELS = [16, 32, 64, 128]
CONV_KERNEL_SIZES = [3, 5]
CONV_STRIDES = [1, 2]
POOLING_KERNEL_SIZES = [2, 3]
POOLING_STRIDES = [2, 3]
LINEAR_UNITS = [128, 256, 512]

# ─── 2. GENOMA BASADO EN GRAFOS (NUEVA ESTRUCTURA) ───────────────────────────

def create_empty_genome() -> Dict[str, Any]:
    """Inicializa un genoma con su estructura de grafo: nodos y aristas."""
    return {"layers": [], "connections": []}

def add_layer_to_genome(genome: Dict[str, Any], layer_gene: Dict[str, Any]) -> int:
    """Añade un nuevo gen de capa al genoma, asignándole un ID único."""
    new_id = len(genome['layers'])
    layer_gene['id'] = new_id
    genome['layers'].append(layer_gene)
    return new_id

def add_connection_to_genome(genome: Dict[str, Any], from_id: int, to_id: int) -> None:
    """Añade una conexión entre dos capas."""
    connection = (from_id, to_id)
    if connection not in genome['connections']:
        genome['connections'].append(connection)

def get_always_valid_kernels(height: int, width: int) -> List[int]:
    # Valid kernel sizes: must be ≤ spatial dims
    max_kernel = min(height, width, max(CONV_KERNEL_SIZES))
    valid_kernels = [k for k in CONV_KERNEL_SIZES if k <= max_kernel]
    if not valid_kernels:
        valid_kernels = [1]
    return valid_kernels

def get_always_valid_strides(height: int, width: int) -> List[int]:
    valid_strides = [
        s for s in CONV_STRIDES
        if (height // s >= 1) and (width // s >= 1)
    ]
    if not valid_strides:
        valid_strides = [1]
    return valid_strides

def get_valid_pooling_kernels_or_return_empty_list(height: int, width: int) -> List[int]:
    min_dim = min(height, width)
    valid_kernels = [k for k in POOLING_KERNEL_SIZES if (min_dim - k) // k + 1 <= min_dim] # assuming kernel_size = strides and dilation = 1
    return valid_kernels

def get_conv_block_gene(input_shape: Tuple[int, int, int]) -> Dict[str, Any]:
    in_ch, h, w = input_shape

    valid_kernels = get_always_valid_kernels(h, w)
    valid_strides = get_always_valid_strides(h, w)
    kernel_size = random.choice(valid_kernels)
    stride = random.choice(valid_strides)
    padding = (kernel_size - 1) // 2  # Same padding

    out_channels = random.choice(CONV_OUT_CHANNELS)

    # Compute output shape after conv
    h_out = (h + 2 * padding - kernel_size) // stride + 1
    w_out = (w + 2 * padding - kernel_size) // stride + 1
    
    # Pooling decision
    pool_type = random.choice(POOLING_TYPES)
    pooling_kernel = None
    if pool_type:
        valid_pooling_kernels = get_valid_pooling_kernels_or_return_empty_list(h_out, w_out)
        if valid_pooling_kernels:
            pooling_kernel = random.choice(valid_pooling_kernels)
            # Compute output shape after pooling
            # We assume kernel_size = strides and dilation = 1
            h_out = (h_out - pooling_kernel) // pooling_kernel + 1
            w_out = (w_out - pooling_kernel) // pooling_kernel + 1
        else:
            pool_type = None

    return {
        'type': 'conv',
        'in_channels': in_ch,
        'out_channels': out_channels,
        'kernel': kernel_size,
        'stride': stride,
        'padding': padding,
        'activation': random.choice(ACTIVATIONS),
        'use_bn': random.choice([True, False]),
        'pool': {'type': pool_type, 'kernel': pooling_kernel} if pool_type else None,
        'output_shape': (out_channels, h_out, w_out)
    }

def get_linear_block_gene(in_features: int) -> Dict[str, Any]:
    out_features = random.choice(LINEAR_UNITS)
    return {
        'type': 'linear',
        'in_features': in_features,
        'out_features': out_features,
        'activation': random.choice(ACTIVATIONS),
        'dropout': random.choice(DROPOUT_PROB),
        'output_shape': (out_features,)
    }

# ─── 3. GENERACIÓN ROBUSTA DE GENOMAS ──────────────────────────────────────────

def random_genome_graph(
    input_shape=(3, 32, 32),
    min_conv_layers=2, max_conv_layers=5,
    min_linear_layers=1, max_linear_layers=2,
    num_classes=10,
    skip_connection_prob=0.3
):
    """
    Construye un genoma de red neuronal como un grafo (DAG).

    Esta función es robusta porque:
      - Rastrea las formas de los tensores para garantizar la validez de las capas.
      - Construye un grafo acíclico dirigido (DAG) por diseño.
      - Puede crear múltiples ramas y unirlas con conexiones de salto.
    """
    genome = create_empty_genome()
    shape_tracker = {} # Diccionario para rastrear la forma de salida de cada capa

    # --- Capa de Entrada ---
    in_channels, h, w = input_shape
    input_id = add_layer_to_genome(genome, {'type': 'input'})
    shape_tracker[input_id] = (in_channels, h, w)
    
    last_conv_layer_id = input_id
    conv_layer_ids = []

    # --- Bloques Convolucionales ---
    num_conv_layers = random.randint(min_conv_layers, max_conv_layers)
    for _ in range(num_conv_layers):
        gene = get_conv_block_gene(
            shape_tracker[last_conv_layer_id]
        )
        new_id = add_layer_to_genome(genome, gene)
        add_connection_to_genome(genome, last_conv_layer_id, new_id)
        shape_tracker[new_id] = gene['output_shape']

        # Skip connections
        if len(conv_layer_ids) > 0 and random.random() < skip_connection_prob:
            # There are no constraints because the shapes will be matched under demand with pytorch on forward pass
            skip_source_id = random.choice(conv_layer_ids)
            add_connection_to_genome(genome, skip_source_id, new_id)

        last_conv_layer_id = new_id
        conv_layer_ids.append(new_id)

    # --- Transición a Capas Lineales (Flatten o Pooling Global) ---
    last_feature_id = last_conv_layer_id
    in_ch, h, w = shape_tracker[last_feature_id]
    
    if random.choice([True, False]):
        gene = {'type': 'global_pool', 'pool_type': 'adaptive_avg'}
        in_features = in_ch
    else:
        gene = {'type': 'flatten'}
        in_features = in_ch * h * w

    transition_id = add_layer_to_genome(genome, gene)
    add_connection_to_genome(genome, last_feature_id, transition_id)
    shape_tracker[transition_id] = (in_features,)
    last_linear_id = transition_id

    # --- Bloques Lineales ---
    num_linear_layers = random.randint(min_linear_layers, max_linear_layers)
    for _ in range(num_linear_layers):
        gene = get_linear_block_gene(in_features)
        new_id = add_layer_to_genome(genome, gene)
        add_connection_to_genome(genome, last_linear_id, new_id)
        
        shape_tracker[new_id] = gene["output_shape"]
        last_linear_id = new_id

    # --- Capa de Salida ---
    last_layer_input_features = gene["output_shape"][0]
    gene = {
        'type': 'linear',
        'in_features': last_layer_input_features,
        'out_features': num_classes,
        'activation': 'none', # La activación final (softmax) se aplica en la función de pérdida
        'dropout': 0.0
    }
    output_id = add_layer_to_genome(genome, gene)
    add_connection_to_genome(genome, last_linear_id, output_id)
    add_layer_to_genome(genome, {'type': 'output', 'from': output_id}) # Nodo final simbólico
    print(shape_tracker)

    return genome

In [8]:
random_genome_graph(
    input_shape=(3, 32, 32),
    min_conv_layers=2, max_conv_layers=5,
    min_linear_layers=1, max_linear_layers=2,
    num_classes=10,
    skip_connection_prob=0.5
)["connections"]

{0: (3, 32, 32), 1: (128, 16, 16), 2: (128, 2, 2), 3: (128, 0, 0), 4: (32, 0, 0), 5: (16, 0, 0), 6: (16,), 7: (128,), 8: (128,)}


[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]

In [306]:
# ─── 4. CONSTRUCTOR DEL MODELO A PARTIR DEL GRAFO ──────────────────────────────

class MergeLayer(nn.Module):
    """Capa para fusionar varias entradas concatenándolas en la dimensión del canal."""
    def forward(self, *inputs):
        # Asegurarse de que todos los tensores de entrada tengan las mismas dimensiones espaciales
        # antes de la concatenación. Este chequeo es opcional pero recomendado.
        # for i in range(1, len(inputs)):
        #     assert inputs[i].shape[2:] == inputs[0].shape[2:], "Las dimensiones espaciales deben coincidir para la fusión"
        return torch.cat(inputs, dim=1)

class GenomeNet(nn.Module):
    def __init__(self, genome):
        super().__init__()
        self.genome = genome
        self.torch_layers = nn.ModuleDict()
        self._build_model()

    def _get_input_ids_for(self, layer_id):
        """Encuentra todos los IDs de capas que conectan a la capa layer_id."""
        return [src for src, dest in self.genome['connections'] if dest == layer_id]

    def _build_model(self):
        """Construye las capas de PyTorch basándose en los genes del genoma."""
        # Un mapa para rastrear el número de canales de entrada después de las fusiones
        in_channels_map = {}

        for gene in self.genome['layers']:
            layer_id = gene['id']
            layer_type = gene.get('type')
            
            # Determinar las entradas para esta capa
            input_ids = self._get_input_ids_for(layer_id)
            
            current_in_channels = 0
            if input_ids:
                # Si hay múltiples entradas, se fusionarán. La entrada total de canales es la suma.
                if len(input_ids) > 1:
                    self.torch_layers[f"merge_{layer_id}"] = MergeLayer()
                    current_in_channels = sum(in_channels_map[i][0] for i in input_ids)
                else: # Una sola entrada
                    current_in_channels = in_channels_map[input_ids[0]][0]

            # Instanciar la capa de PyTorch correspondiente
            if layer_type == 'conv':
                gene['in_channels'] = current_in_channels
                block = self._create_conv_block(gene)
                self.torch_layers[str(layer_id)] = block
                in_channels_map[layer_id] = (gene['out_channels'],)

            elif layer_type == 'linear':
                block = self._create_linear_block(gene)
                self.torch_layers[str(layer_id)] = block
                in_channels_map[layer_id] = (gene['out_features'],)
                
            elif layer_type == 'flatten':
                self.torch_layers[str(layer_id)] = nn.Flatten()
                # La forma se determina dinámicamente, no es necesario almacenarla aquí.
                
            elif layer_type == 'global_pool':
                pool_size = (1, 1)
                if gene['pool_type'] == 'adaptive_avg':
                    self.torch_layers[str(layer_id)] = nn.Sequential(nn.AdaptiveAvgPool2d(pool_size), nn.Flatten())
                elif gene['pool_type'] == 'adaptive_max':
                    self.torch_layers[str(layer_id)] = nn.Sequential(nn.AdaptiveMaxPool2d(pool_size), nn.Flatten())
                in_channels_map[layer_id] = (current_in_channels,)
            
            elif layer_type == 'input':
                # El nodo de entrada no tiene capa de PyTorch; solo es un punto de partida.
                in_channels_map[layer_id] = (self.genome['layers'][0].get('shape', (3, 32, 32))[0],)

    def _create_conv_block(self, gene):
        layers = []
        layers.append(nn.Conv2d(
            in_channels=gene['in_channels'], out_channels=gene['out_channels'],
            kernel_size=gene['kernel'], stride=gene['stride'], padding=gene['kernel'] // 2
        ))
        if gene['use_bn']:
            layers.append(nn.BatchNorm2d(gene['out_channels']))
        if gene['activation'] == 'relu':
            layers.append(nn.ReLU(inplace=True))
        elif gene['activation'] == 'leaky_relu':
            layers.append(nn.LeakyReLU(inplace=True))
        elif gene['activation'] == 'gelu':
            layers.append(nn.GELU())
        
        pool = gene.get('pool')
        if pool:
            pool_type = pool.get('type')
            if pool_type == 'max':
                pool_kernel = gene.get('pool').get('kernel')
                layers.append(nn.MaxPool2d(kernel_size=pool_kernel))
            elif pool_type == 'avg':
                pool_kernel = gene.get('pool').get('kernel')
                layers.append(nn.AvgPool2d(kernel_size=pool_kernel))
        return nn.Sequential(*layers)

    def _create_linear_block(self, gene):
        layers = []
        layers.append(nn.Linear(gene['in_features'], gene['out_features']))
        if gene['activation'] == 'relu':
            layers.append(nn.ReLU(inplace=True))
        elif gene['activation'] == 'leaky_relu':
            layers.append(nn.LeakyReLU(inplace=True))
        elif gene['activation'] == 'gelu':
            layers.append(nn.GELU())
        if gene['dropout'] > 0:
            layers.append(nn.Dropout(p=gene['dropout']))
        return nn.Sequential(*layers)

    def forward(self, x):
        # Diccionario para almacenar las salidas de cada capa
        outputs = {}
        
        # El nodo 0 es la entrada
        outputs[0] = x
        
        # Iterar a través de las capas en orden de ID (garantiza el orden topológico)
        for layer_id_str, torch_layer in self.torch_layers.items():
            if "merge" in layer_id_str:
                continue # Los merges se manejan bajo demanda
            
            layer_id = int(layer_id_str)
            input_ids = self._get_input_ids_for(layer_id)
            
            # Recopilar tensores de entrada
            input_tensors = [outputs[i] for i in input_ids]
            
            # Fusionar si es necesario
            if len(input_tensors) > 1:
                merge_layer = self.torch_layers[f"merge_{layer_id}"]
                model_input = merge_layer(*input_tensors)
            else:
                model_input = input_tensors[0]

            # Pasar por la capa actual
            outputs[layer_id] = torch_layer(model_input)
            
        # El último nodo del genoma es el de tipo 'output'
        output_node = next(g for g in self.genome['layers'] if g.get('type') == 'output')
        final_tensor = outputs[output_node['from']]
        return final_tensor

In [307]:
# ─── 5. ALGORITMO GENÉTICO: MUTACIÓN Y CRUCE ───────────────────────────────

def mutate_genome(genome, mutation_rate=0.1):
    """
    Aplica mutaciones a un genoma. Tipos de mutación:
    1. Mutación de hiperparámetros de una capa.
    2. Adición de una nueva conexión de salto (skip connection).
    """
    new_genome = copy.deepcopy(genome) # Trabajar sobre una copia
    
    # 1. Mutar hiperparámetros de capas existentes
    for gene in new_genome['layers']:
        if random.random() < mutation_rate:
            if gene['type'] == 'conv':
                gene['kernel'] = random.choice([3, 5])
                gene['activation'] = random.choice(ACTIVATIONS)
            elif gene['type'] == 'linear' and gene.get('activation') != 'none':
                gene['dropout'] = random.choice(DROPOUT_PROB)
                gene['activation'] = random.choice(ACTIVATIONS)
    
    # 2. Añadir una nueva conexión de salto
    if random.random() < mutation_rate:
        conv_layers = [g for g in new_genome['layers'] if g['type'] == 'conv']
        if len(conv_layers) >= 2:
            source_gene = random.choice(conv_layers)
            dest_gene = random.choice(conv_layers)
            
            # Asegurar que la conexión sea hacia adelante y no inmediata
            if source_gene['id'] < dest_gene['id'] - 1:
                # Este es un intento simple; un sistema real recalcularía las formas
                # para asegurar la compatibilidad o añadiría capas de adaptación.
                # Por ahora, simplemente añadimos la conexión. La robustez del
                # constructor del modelo (con MergeLayer) lo manejará.
                print(f"MUTATION: Adding skip connection from {source_gene['id']} to {dest_gene['id']}")
                add_connection_to_genome(new_genome, source_gene['id'], dest_gene['id'])

    return new_genome

def crossover_genomes(parent1, parent2):
    """
    Realiza un cruce entre dos genomas padres.
    Estrategia: Tomar la base convolucional de un padre y el clasificador del otro.
    """
    child_genome = create_empty_genome()
    
    # Encontrar el punto de transición (flatten o global_pool) en el padre 1
    p1_transition_idx = -1
    for i, gene in enumerate(parent1['layers']):
        if gene['type'] in ['flatten', 'global_pool']:
            p1_transition_idx = i
            break
    if p1_transition_idx == -1: return copy.deepcopy(parent1) # No se pudo cruzar

    # Copiar la parte convolucional del padre 1
    for i in range(p1_transition_idx + 1):
        add_layer_to_genome(child_genome, copy.deepcopy(parent1['layers'][i]))
    for conn in parent1['connections']:
        if conn[1] <= p1_transition_idx:
            add_connection_to_genome(child_genome, conn[0], conn[1])

    last_child_id = p1_transition_idx
    
    # Encontrar la forma de salida de la parte convolucional
    # Para este ejemplo, asumiremos que se puede calcular o es conocida.
    # Una implementación más avanzada recalcularía la forma exacta.
    # Aquí lo simulamos buscando el primer lineal en el padre 2 para obtener las features.
    p2_first_linear = next((g for g in parent2['layers'] if g['type'] == 'linear'), None)
    if not p2_first_linear: return copy.deepcopy(parent1)

    in_features = p2_first_linear['in_features']
    # En un sistema real, se recalcularía la salida de la capa last_child_id
    # y se ajustaría in_features de la primera capa lineal del hijo.
    
    # Copiar la parte lineal (clasificador) del padre 2
    p2_linear_genes = [g for g in parent2['layers'] if g['type'] in ['linear', 'output']]
    
    id_map = {} # Mapeo de IDs antiguos (p2) a nuevos (hijo)
    
    # Conectar la parte conv del hijo con la parte lineal del padre 2
    first_linear_gene = copy.deepcopy(p2_linear_genes[0])
    first_linear_gene['in_features'] = in_features # Ajuste (simplificado)
    new_id = add_layer_to_genome(child_genome, first_linear_gene)
    id_map[p2_linear_genes[0]['id']] = new_id
    add_connection_to_genome(child_genome, last_child_id, new_id)
    last_child_id = new_id

    for i in range(1, len(p2_linear_genes)):
        gene = copy.deepcopy(p2_linear_genes[i])
        old_id = gene['id']
        
        # Si es el nodo 'output', solo se ajusta su referencia 'from'
        if gene['type'] == 'output':
            gene['from'] = id_map[gene['from']]
            add_layer_to_genome(child_genome, gene)
            continue

        new_id = add_layer_to_genome(child_genome, gene)
        id_map[old_id] = new_id
        
        # Encontrar la conexión original en el padre 2 y recrearla con nuevos IDs
        original_conn_source = next(s for s, d in parent2['connections'] if d == old_id)
        add_connection_to_genome(child_genome, id_map[original_conn_source], new_id)

    print("CROSSOVER: Created a new child genome.")
    return child_genome

In [308]:
# ─── 6. EJECUCIÓN: DEMOSTRACIÓN COMPLETA ──────────────────────────────────────

if __name__ == '__main__':
    # (1) Generar un genoma aleatorio (Padre 1)
    print("="*20 + " PADRE 1 " + "="*20)
    genome1 = random_genome_graph(input_shape=(3, 32, 32), num_classes=10)
    print("--- GENOMA (Grafo) ---")
    pprint.pprint(genome1)
    
    # (2) Construir el modelo a partir del genoma
    model1 = GenomeNet(genome1)
    print("\n--- ARQUITECTURA DEL MODELO ---")
    print(model1)
    
    # Probar con un tensor de entrada aleatorio
    try:
        dummy_input = torch.randn(1, 3, 32, 32)
        output = model1(dummy_input)
        print(f"\nPrueba de forward exitosa. Forma de salida: {output.shape}")
    except Exception as e:
        print(f"\nError durante la prueba de forward: {e}")

    # (3) Demostración de Mutación
    print("\n" + "="*20 + " MUTACIÓN " + "="*20)
    mutated_genome = mutate_genome(genome1)
    print("--- GENOMA MUTADO ---")
    pprint.pprint(mutated_genome)
    mutated_model = GenomeNet(mutated_genome)
    print("\n--- MODELO MUTADO ---")
    print(mutated_model)

    # (4) Demostración de Cruce
    print("\n" + "="*20 + " CRUCE " + "="*20)
    # Generar un segundo padre
    print("--- Generando Padre 2 ---")
    genome2 = random_genome_graph(input_shape=(3, 32, 32), num_classes=10)
    # pprint.pprint(genome2)
    
    child_genome = crossover_genomes(genome1, genome2)
    print("\n--- GENOMA HIJO ---")
    pprint.pprint(child_genome)
    child_model = GenomeNet(child_genome)
    print("\n--- MODELO HIJO ---")
    print(child_model)

==================== PADRE 1 ====================
{0: (3, 32, 32), 1: (32, 16, 16), 2: (16, 8, 8), 3: (16,), 4: (512,)}
--- GENOMA (Grafo) ---
{'connections': [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)],
 'layers': [{'id': 0, 'type': 'input'},
            {'activation': 'relu',
             'id': 1,
             'in_channels': 3,
             'kernel': 3,
             'out_channels': 32,
             'output_shape': (32, 16, 16),
             'padding': 1,
             'pool': None,
             'stride': 2,
             'type': 'conv',
             'use_bn': True},
            {'activation': 'relu',
             'id': 2,
             'in_channels': 32,
             'kernel': 3,
             'out_channels': 16,
             'output_shape': (16, 8, 8),
             'padding': 1,
             'pool': None,
             'stride': 2,
             'type': 'conv',
             'use_bn': True},
            {'id': 3, 'pool_type': 'adaptive_avg', 'type': 'global_pool'},
            {'activation': 

In [315]:
genome1 = random_genome_graph(
    input_shape=(3, 64, 64),
    min_conv_layers=2, 
    max_conv_layers=5,
    min_linear_layers=1,
    max_linear_layers=2,
    num_classes=10,
    skip_connection_prob=0.0
)
model1 = GenomeNet(genome1)
print(model1)

x = torch.randn(12, 3, 64, 64)
model1(x)

{0: (3, 64, 64), 1: (64, 32, 32), 2: (32, 8, 8), 3: (32,), 4: (512,), 5: (512,)}
GenomeNet(
  (torch_layers): ModuleDict(
    (1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): AdaptiveAvgPool2d(output_size=(1, 1))
      (1): Flatten(start_dim=1, end_dim=-1)
    )
    (4): Sequential(
      (0): Linear(in_features=32, out_features=512, bias=True)
      (1): ReLU(inplace=True)
    )
    (5): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): GELU(approximate='none')
    )
    (6): Sequential(
      (0): Linear(in

tensor([[ 0.0109, -0.0070, -0.0120, -0.0877, -0.0277,  0.0397, -0.0256, -0.0249,
          0.0071,  0.0037],
        [ 0.0121, -0.0092, -0.0113, -0.0868, -0.0274,  0.0459, -0.0276, -0.0236,
          0.0081,  0.0047],
        [ 0.0116, -0.0120, -0.0141, -0.0894, -0.0318,  0.0473, -0.0256, -0.0240,
          0.0093,  0.0048],
        [ 0.0156, -0.0076, -0.0116, -0.0856, -0.0299,  0.0398, -0.0257, -0.0188,
          0.0098,  0.0023],
        [ 0.0153, -0.0094, -0.0154, -0.0860, -0.0291,  0.0425, -0.0233, -0.0192,
          0.0091,  0.0061],
        [ 0.0156, -0.0072, -0.0149, -0.0875, -0.0292,  0.0438, -0.0259, -0.0199,
          0.0085,  0.0034],
        [ 0.0142, -0.0095, -0.0115, -0.0888, -0.0287,  0.0411, -0.0280, -0.0212,
          0.0094,  0.0062],
        [ 0.0115, -0.0068, -0.0135, -0.0859, -0.0269,  0.0434, -0.0258, -0.0217,
          0.0078,  0.0031],
        [ 0.0141, -0.0106, -0.0138, -0.0877, -0.0289,  0.0406, -0.0260, -0.0199,
          0.0066,  0.0040],
        [ 0.0125, -

In [39]:
model1

GenomeNet(
  (torch_layers): ModuleDict(
    (1): Sequential(
      (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
    (4): Sequential(
      (0): Conv2d(32, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (5): Sequential(
      (0): Conv2d(128, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    

In [29]:
model1

GenomeNet(
  (torch_layers): ModuleDict(
    (1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): GELU(approximate='none')
    )
    (2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (3): Sequential(
      (0): AdaptiveAvgPool2d(output_size=(1, 1))
      (1): Flatten(start_dim=1, end_dim=-1)
    )
    (4): Sequential(
      (0): Linear(in_features=32, out_features=512, bias=True)
      (1): GELU(approximate='none')
    )
    (5): Sequential(
      (0): Linear(in_features=512, out_features=10, bias=True)
    )
  )
)